In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf

In [2]:
import sys
from pathlib import Path
print(Path.cwd())


/Users/vaishnukanna/Documents/jegadeesh-titman-momentum/results


In [3]:
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [4]:
from src.capm_reg import capm_regression
from src.config import DATA_DIR

In [19]:
path1 = DATA_DIR / "portfolio" / "J6_K6_skip0_returns.csv"
if not path1.exists():
        raise FileNotFoundError(f"Portfolio data not found at {path1}. Run portfolio.py first.")
else:
    portfolio = pd.read_csv(
        path1, 
        parse_dates=["Date"]
        ).set_index("Date")
    print(portfolio.head())

              winner     loser    spread
Date                                    
2001-02-01  0.019540 -0.192802  0.212342
2001-03-01 -0.043318 -0.035796 -0.007522
2001-04-01  0.077074  0.216275 -0.139201
2001-05-01  0.038592 -0.010052  0.048643
2001-06-01  0.033504  0.051841 -0.018337


In [21]:
path2 = DATA_DIR / "portfolio" / "market_returns.csv"
if not path2.exists():
    raise FileNotFoundError(f"Risk-free rate data not found at {path2}. Run capm_params.py first.")
else:
    market = pd.read_csv(
        path2,
        parse_dates=["Date"]
        ).set_index("Date")["Market_Returns"]

    print(market.head())

Date
2000-02-01   -0.020108
2000-03-01    0.096720
2000-04-01   -0.030796
2000-05-01   -0.021915
2000-06-01    0.023934
Name: Market_Returns, dtype: float64


In [ ]:
path3 = DATA_DIR / "portfolio" /"rf_rate.csv"
if not path3.exists():
    raise FileNotFoundError(f"Risk-free rate data not found at {path3}. Run capm_params.py first.")
else:
    rf = pd.read_csv(
        path3,
        parse_dates=["Date"]
    ).set_index("Date")["Rf_Rate"]

    print(rf.head())

Date
2000-01-01    0.004792
2000-02-01    0.004867
2000-03-01    0.004883
2000-04-01    0.004833
2000-05-01    0.004950
Name: Rf_Rate, dtype: float64

In [8]:
data = pd.concat(
    [portfolio, market, rf],
    axis=1
).dropna()

data.head()

,winner,loser,spread,Market_Returns,Rf_Rate
Date,,,,,
2001-02-01,0.019540,-0.192802,0.212342,-0.092291,0.003758
2001-03-01,-0.043318,-0.035796,-0.007522,-0.064205,0.003425
2001-04-01,0.077074,0.216275,-0.139201,0.076814,0.003167
2001-05-01,0.038592,-0.010052,0.048643,0.005090,0.003000
2001-06-01,0.033504,0.051841,-0.018337,-0.025035,0.002808


In [9]:
winner = data["winner"]
loser = data["loser"]
spread = data["spread"]
market = data["Market_Returns"]
rf = data["Rf_Rate"]

In [10]:
print(type(winner))
print(type(market))
print(type(rf))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [11]:
capm_winner = capm_regression(
    portfolio_returns=winner,
    market_returns=market,
    risk_free_rate=rf
)

capm_winner

const    0.017649
Rm       1.016668
dtype: float64
Index(['const', 'Rm'], dtype='object')


{'alpha': np.float64(0.017649203034305096),
 'beta': np.float64(1.0166677567703388),
 't_alpha': np.float64(9.109240039839353),
 't_beta': np.float64(25.02863283146),
 'r_squared': np.float64(0.7635393415978341)}

In [12]:
capm_loser = capm_regression(
    portfolio_returns=loser,
    market_returns=market,
    risk_free_rate=rf
)

capm_loser

const    0.001316
Rm       1.500632
dtype: float64
Index(['const', 'Rm'], dtype='object')


{'alpha': np.float64(0.0013160441922840794),
 'beta': np.float64(1.5006319057531274),
 't_alpha': np.float64(0.4644166857753696),
 't_beta': np.float64(25.258790315738686),
 'r_squared': np.float64(0.7668287414244354)}

In [13]:
capm_strategy = capm_regression(
    portfolio_returns=spread,
    market_returns=market,
    risk_free_rate=rf
)

capm_strategy

const    0.015346
Rm      -0.480876
dtype: float64
Index(['const', 'Rm'], dtype='object')


{'alpha': np.float64(0.015346161067325143),
 'beta': np.float64(-0.48087597834468965),
 't_alpha': np.float64(4.190055477400894),
 't_beta': np.float64(-6.262592214442643),
 'r_squared': np.float64(0.16816761403427405)}

In [14]:
capm_table = pd.DataFrame({
    "Winner": capm_winner,
    "Loser": capm_loser,
    "Winner-Loser": capm_strategy
}).T

In [15]:
capm_table

,alpha,beta,t_alpha,t_beta,r_squared
Winner,0.017649,1.016668,9.109240,25.028633,0.763539
Loser,0.001316,1.500632,0.464417,25.258790,0.766829
Winner-Loser,0.015346,-0.480876,4.190055,-6.262592,0.168168


In [16]:
capm_table.to_csv(PROJECT_ROOT/"results/capm.csv")